In [21]:
#Reference

#https://python.langchain.com/docs/tutorials/rag/
#https://www.youtube.com/watch?v=yF9kGESAi3M&t=3823s


#The RAG has two main components

#Indexing - taking data from source and indexing it , happens offline
# in other words ,
# (Document Loaders) take/load data
# (Text Splitters) break it down to chunks
# (Embedding model)convert the chunks to embeddings
# (Vector store)store the data inform of embeddings

#Retrieval- the RAG chain, takes user query at runtime, retrieves relevant data from index, then passes to the model
# will be in more detail in RAG part b



In [22]:
# select chatmodel
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from textdistance import overlap

load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model("gpt-4o-mini", model_provider="openai")


In [23]:
#having issue with openai embedding
#from langchain_openai import OpenAIEmbeddings
#embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

#replaced it with hugging face embedding

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [17]:
#tried using vector stores,inmemory, had issues when i tried to implement it like this
# hence used FAISS
#from langchain_core.vectorstores import InMemoryVectorStore
#vector_store=InMemoryVectorStore(embeddings)


In [18]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
# we will receive a html file
# we are going to select only title, header and content  from the html
bs4_strainer=bs4.SoupStrainer(class_=("post-title","post-header","post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs=loader.load()
assert len(docs)==1
print(f"Total document length(characters) {len(docs[0].page_content)}")

Total document length(characters) 43130


In [19]:
#spliting the document for vector storage
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)
all_splits=text_splitter.split_documents(docs)
print(len(all_splits))
print(all_splits[0])
#print(len(all_splits[65]))


66
page_content='LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory' metadata={'s

In [20]:

from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(documents=all_splits, embedding=embeddings)
#storing documents
document_ids=vector_store.add_documents(documents=all_splits)
print(document_ids[0:5])

['442dd2f0-d4ad-4feb-a19a-93c500e18364', 'cc2acfd3-3484-4b62-92f8-2e91baafb755', 'edf331ca-29c0-4666-be47-0f219d4cdb33', 'aa05f05c-e3fd-4893-9307-23dd1a2b0787', 'e7f253c2-9f38-44ba-9fb6-a96f3f98f35a']
